In [1]:
import torch, torchattacks
from torchvision import datasets, models, transforms

In [2]:
# https://docs.pytorch.org/vision/main/generated/torchvision.datasets.ImageNet.html
# https://image-net.org/challenges/LSVRC/2012/2012-downloads.php

Step 1: Load three different pretrained models from PyTorch. There are several pretrained models available.

In [3]:
torch.hub.set_dir(".\models")
alexnet  = torch.hub.load("pytorch/vision", model="alexnet",  weights="AlexNet_Weights.IMAGENET1K_V1")
resnet50 = torch.hub.load("pytorch/vision", model="resnet50", weights="ResNet50_Weights.IMAGENET1K_V1")
vit      = torch.hub.load("pytorch/vision", model="vit_b_16", weights="ViT_B_16_Weights.IMAGENET1K_V1")

Using cache found in .\models\pytorch_vision_main
Using cache found in .\models\pytorch_vision_main
Using cache found in .\models\pytorch_vision_main


Step 2: In class we discussed a few adversarial attack techniques such as PGD, FGSM, etc.

torchattacks is a framework which lets you attack any model with many such adversarial attack techniques. Install torchattacks and then use it to attack your loaded model with 3 different attack techniques.

In [20]:
# imagenet = datasets.ImageNet(root = ".\data", split = 'eval', transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ]))
dataset = datasets.CIFAR10(root = ".\data", train=False,  transform = transforms.Compose([
    # transforms.Resize((229)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]), download = True)
images = []
labels = []
for x, y in dataset:
    images.append(x)
    labels.append(y)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

In [ ]:
# https://adversarial-attacks-pytorch.readthedocs.io/en/latest/index.html
# https://github.com/Harry24k/adversarial-attacks-pytorch
# target_labels = [579 for _ in range(len(dataset.classes))]
target_labels = [0 for _ in range(len(dataset.classes))]

for model_idx, model in enumerate([resnet50, alexnet, vit]):
    attk_PGD  = torchattacks.PGD (model, eps=8/255, alpha=1/255, steps=10, random_start=True)
    attk_FGSM = torchattacks.FGSM(model, eps=8/255)
    attk_CW   = torchattacks.CW  (model, c=1, kappa=0, steps=50, lr=0.01)

    for attk_idx, attk in enumerate([attk_PGD, attk_FGSM, attk_CW]):
        attk.set_normalization_used(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        attk.set_mode_targeted_by_label(quiet=True)
        adv_images = attk(images, target_labels)
        attk.save(adv_images, save_path=f".\adv_images\data_model{model_idx}_attk{attk_idx}.pt", verbose=True)


TypeError: unsupported operand type(s) for *: 'numpy.ndarray' and 'Tensor'

Step 3: Report accuracy in a 3×3 table (3 rows for models and 3 columns for attack techniques)

Step 4: Choose any image of your choice and show the corresponding adversarial images generated by each attack technique from Step 2. 

For each adversarial image, report the average mean-squared error between the original image and the adversariaal image.